<a href="https://colab.research.google.com/github/ev1025/Gemma2_FineTuning_Math_Teacher_Mingle/blob/main/Math_Teacher_Mingle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Math Teacher Mingle

In [ ]:
#허깅페이스 설치
!pip install datasets huggingface_hub

In [ ]:
#Unsloth 설치

%%capture
# Installs Unsloth, Xformers (Flash Attention) and all other packages!
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

# We have to check which Torch version for Xformers (2.3 -> 0.0.27)
from torch import __version__; from packaging.version import Version as V
xformers = "xformers==0.0.27" if V(__version__) < V("2.4.0") else "xformers"
!pip install --no-deps {xformers} trl peft accelerate bitsandbytes triton

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from huggingface_hub import login
from google.colab import userdata

import os
import torch
from datasets import Dataset, load_dataset, concatenate_datasets
import pandas as pd

from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import FastLanguageModel, is_bfloat16_supported, UnslothTrainer, UnslothTrainingArguments

output_dir = '/content/drive/MyDrive/outputs'

#허깅페이스 액세스토큰
hf_token = userdata.get('hf_token')

# 허깅페이스 로그인
login(hf_token)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
max_seq_length = 2048 # 모델이 입력으로 받을 수 있는 최대 입력 시퀀스 (길면 더 많은 메모리 소모)
dtype = torch.float16 # None 자동 / torch.float16(gpu- Teslcolab-newa T4, V100,) / Bfloat16 (gpu - Ampere+,A100, RTX 30xx 이상)
load_in_4bit = True   # 양자화(메모리 사용량 축소를 위해 쪼개기)

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/gemma-2-2b-bnb-4bit", # 모델 선택
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,                # 양자화(PEFT모델 init_lora_weights='loftq'와 중복불가능)
    token = hf_token
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
==((====))==  Unsloth 2024.9: Fast Gemma2 patching. Transformers = 4.44.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/2.22G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/46.4k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

### PEFT Model(모델의 미세 파라미터 조정)

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,            # LoRA의 랭크 값, 메모리사용량 증가, 모델의 표현력 좋아짐 (8, 16, 32, 64, 128), 메모리 영향 있음
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj","embed_tokens",],
                      # "embed_tokens", "lm_head",], # 포함하면 파라미터 수 폭증, 메모리 영향 있음
    lora_alpha = 32,  # Lora의 스케일링, 학습성능 개선될 수 있음, r의 2배수로 하는 게 좋음 메모리 영향 있음
    lora_dropout = 0, # 과적합 방지, 0이 최적화
    bias = "none",    # Supports any, but = "none" is optimized
    use_gradient_checkpointing = "unsloth", # "unsloth"나 True로 설정하면 긴 컨텍스트 처리시 메모리 30% 절약, 메모리 영향 있음
    random_state = 3407,
    use_rslora = True,       # 안정화된 랭크를 사용할지
    loftq_config = None,     # 로라 양자화 안함(하고싶으면 아래 init_lora 켜기, ), 메모리 영향 있음
    # init_lora_weights='loftq'# 양자화로 모델의 메모리 사용량 줄이고 추론 속도를 빠르게 함(Pretrained모델의 load_in_4bit와 중복 안 됨)
)

Unsloth: Offloading input_embeddings to disk to save VRAM


/usr/local/lib/python3.10/dist-packages/unsloth/models/_utils.py:866: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  offloaded_W = torch.load(filename, map_location = "cpu", 

Unsloth: Casting embed_tokens to float32


In [ ]:
# 로라 파라미터 몇% 학습됐는지(trainable%:)
model.print_trainable_parameters()

trainable params: 610,590,720 || all params: 3,814,756,608 || trainable%: 16.0060


### 세부 데이터 업로드(Fine tuning data upload)

In [ ]:
math_data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/math_data.csv')
math_data.head()

In [ ]:
# 열 이름 변경: input -> instruction, response -> out
math_data = math_data.rename(columns={'input': 'instruction', 'response': 'output'})

# pandas DataFrame을 Dataset으로 변환
math_data_prep = Dataset.from_pandas(math_data)
math_data_prep = math_data_prep.add_column("url", [""] * len(math_data_prep)) # alphako 형식맞추기 위해 url열 추가

# 변환된 데이터셋 확인
print(math_data_prep)

In [ ]:
#hugging face에 수학 데이터 업로드
math_data_prep.push_to_hub("Envy1025/mathdata")

### 데이터 불러오기

In [ ]:
alpaca_dataset = load_dataset("beomi/KoAlpaca-v1.1a",  split = "train") # "train[10%]" 이렇게 쓰면 데이터 10%만 가져옴
math_dataset = load_dataset("Envy1025/mathdata",  split = "train")

# CPT를 위해 두 데이터 MERGE
dataset = concatenate_datasets([alpaca_dataset,math_dataset])
print(dataset)

# alpaca_dataset = alpaca_dataset.train_test_split(train_size=0.01)["train"] 너무 많으면 1%데이터만

README.md:   0%|          | 0.00/1.75k [00:00<?, ?B/s]

(…)-00000-of-00001-21df739eb88d711e.parquet:   0%|          | 0.00/12.9M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/21155 [00:00<?, ? examples/s]

README.md:   0%|          | 0.00/339 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/8.95k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/75 [00:00<?, ? examples/s]

Dataset({
    features: ['instruction', 'output', 'url'],
    num_rows: 21230
})


In [ ]:
dataset[0]['output']

'양파는 잎이 아닌 식물의 줄기 부분입니다. 고구마는 식물의 뿌리 부분입니다. \n\n식물의 부위의 구분에 대해 궁금해하는 분이라면 분명 이 질문에 대한 답을 찾고 있을 것입니다. 양파는 잎이 아닌 줄기 부분입니다. 고구마는 다른 질문과 답변에서 언급된 것과 같이 뿌리 부분입니다. 따라서, 양파는 식물의 줄기 부분이 되고, 고구마는 식물의 뿌리 부분입니다.\n\n 덧붙이는 답변: 고구마 줄기도 볶아먹을 수 있나요? \n\n고구마 줄기도 식용으로 볶아먹을 수 있습니다. 하지만 줄기 뿐만 아니라, 잎, 씨, 뿌리까지 모든 부위가 식용으로 활용되기도 합니다. 다만, 한국에서는 일반적으로 뿌리 부분인 고구마를 주로 먹습니다.'

### 프롬프트 형식

In [ ]:
_alpaca_prompt = """Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
{}

### output:
{}"""
# Becomes:
alpaca_prompt = """다음은 작업을 설명하는 명령입니다. 요청을 적절하게 완료하는 응답을 작성하세요.

### 지침:
{}

### 응답:
{}"""

EOS_TOKEN = tokenizer.eos_token # output의 끝을 알리는 토큰

def formatting_prompts_func(examples):
    texts = []
    for instruction, output in zip(examples['instruction'], examples['output']):
            text = alpaca_prompt.format(instruction, output) + EOS_TOKEN
            texts.append(text)
    return { "text" : texts, }
pass

In [ ]:
# 데이터 형태를 프롬프트 형태로 변경
dataset = dataset.map(formatting_prompts_func, batched = True,)

Map:   0%|          | 0/21230 [00:00<?, ? examples/s]

### Continued Pre-Training(CPT)

https://huggingface.co/transformers/v4.2.2/main_classes/optimizer_schedules.html#transformers.SchedulerType

In [ ]:
trainer = UnslothTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length, # 최대 시퀀스 길이
    dataset_num_proc = 8, # 데이터 처리에 사용할 프로세스
    packing=False,        # 짧은 시퀀스에 대한 학습 속도를 5배 빠르게 할 수 있음

    args = UnslothTrainingArguments(
        per_device_train_batch_size = 2, # 각 디바이스 훈련 배치 크기 / 메모리 영향 있음
        gradient_accumulation_steps = 8, # 메모리 영향 있음
        max_steps = 120,         # epoch와 비슷한거
        # warmup_steps = 10,     # 수행할 워밍업 단계 수
        # warmup_ratio = 0.1,    # 학습률 최적화 파라미터(0.1 = 초반 10%는 점진적으로 학습률 상승하다가 90%는 일정하게 상승)
        # num_train_epochs = 1,  # 훈련 반복 수

        # Select a 2 to 10x smaller learning rate for the embedding matrices!
        learning_rate = 5e-5,           # fine tuning 학습률
        embedding_learning_rate = 1e-5, # pretrain과 fine tuning 사이의 학습에 대한 학습률

        fp16 = True, #  not is_bfloat16_supported() / 메모리 영향 있음
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit", # 최적화 알고리즘
        weight_decay = 0.00,  # 가중치 감소
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

Map (num_proc=8):   0%|          | 0/21230 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


### SFT모델

In [ ]:
# # 파인튜닝을 위한 트레이너 셋팅
# trainer = SFTTrainer(
#     model = model,
#     tokenizer = tokenizer,          # 변환용 토크나이저
#     train_dataset = alpaca_dataset,
#     dataset_text_field = "text",    # 데이터셋에서 텍스트필드 이름
#     max_seq_length = max_seq_length,# 최대 토큰수
#     dataset_num_proc = 2,           # 전처리 프로세스 수
#     packing = False,                # 동시 처리용 - 현재 쓸모 없음
#     # 트레이닝 인자
#     args = TrainingArguments(
#         per_device_train_batch_size = 2, # 각 디바이스당 훈련 배치 크기
#         gradient_accumulation_steps = 4, # 그라디언트 누적 단계 (learning_rate / warmup_steps) * step = 현재 스탭의 학습률(초기 학습률을 낮게 설정.)
#         warmup_steps = 5,     # 웜업 스탭 수
#         # num_train_epochs = 3, # 데이터셋을 몇번 학습할건지 epoch
#         max_steps = 30,      # 최대 몇 스탭까지 공부할 것인가?
#         logging_steps = 1,    # 몇 스탭마다 기록할지.
#         learning_rate = 2e-4, # 학습률
#         fp16 = None,         # fp16 사용 여부, bf16이 지원되지 않는 경우에만 사용 is_bfloat16_supported()
#         bf16= torch.cuda.is_bf16_supported(),# bf16 사용 여부, bf16이 지원되는 경우에만 사용
#         optim = "adamw_8bit",                # 최적화 알고리즘
#         weight_decay = 0.01,                 # 가중치 감소
#         lr_scheduler_type = "linear",        # 학습률 스케줄러 유형 "cosine"도 있음
#         seed = 3407,                         # 랜덤시드임
#         output_dir = output_dir,             # 저장위치
#     ),
# )

### 모델 학습

In [ ]:
# 학습 체크포인트 폴더 유무를 체크합니다.
if not os.path.exists(output_dir):
    trainer_stats = trainer.train(resume_from_checkpoint = True)
else:
    trainer_stats = trainer.train() # 모델을 훈련시키고 통계를 반환

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 21,230 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 8
\        /    Total batch size = 16 | Total steps = 120
 "-____-"     Number of trainable parameters = 610,590,720


Unsloth: Setting lr = 1.00e-05 instead of 5.00e-05 for embed_tokens.


Step,Training Loss
1,2.122900
2,2.179400
3,1.883200
4,1.927200
5,1.799600
6,1.773300
7,1.689300
8,1.654200
9,1.650300
10,1.777700


## 모델 테스트

In [ ]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        # "Continue the fibonacci sequence: 1, 1, 2, 3, 5, 8,", # instruction
        "피보나치 수열을 계속하세요: 1, 1, 2, 3, 5, 8,", # instruction
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
tokenizer.batch_decode(outputs)

['<bos>다음은 작업을 설명하는 명령입니다. 요청을 적절하게 완료하는 응답을 작성하세요.\n\n### 지침:\n피보나치 수열을 계속하세요: 1, 1, 2, 3, 5, 8,\n\n### 응답:\n피보나치 수열은 1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 233, 377, 610']

In [ ]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        # "What is Korean music like?"
        "거듭제곱이 무엇인가요?", # instruction
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

<bos>다음은 작업을 설명하는 명령입니다. 요청을 적절하게 완료하는 응답을 작성하세요.

### 지침:
거듭제곱이 무엇인가요?

### 응답:
거듭제곱은 숫자를 곱해도 곱해도 같은 결과를 얻을 수 있는 방법입니다. 예를 들어, 2^3 = 8, 2^4 = 16, 2^5 = 32, 2^6 = 64, 2^7 = 128, 2^8 = 256, 2^9 = 512, 2^10 = 1024, 2^11 = 2048, 2^12 = 4


In [ ]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        # "What is Korean music like?"
        "최소공배수가 무엇인가요?", # instruction
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

<bos>다음은 작업을 설명하는 명령입니다. 요청을 적절하게 완료하는 응답을 작성하세요.

### 지침:
최소공배수가 무엇인가요?

### 응답:
최소공배수는 두 수의 최대공약수와 그 최대공약수의 배수를 합한 수입니다. 예를 들어, 10과 15의 최소공배수는 30입니다. 10과 15의 최대공약수는 5입니다. 따라서, 10과 15의 최소공배수는 5 x 30 = 150입니다.<eos>


## 모델 저장

양자화 https://github.com/unslothai/unsloth/wiki#gguf-quantization-options

In [ ]:
# 로컬(구글드라이브) 저장
# gguf 파라미터4비트 모델이므로 이 방식이 적합
quantization_method = "q4_k_m"   # "q8_0"  "f16" "q8_0" "q4_k_m" "q5_k_m"
model.save_pretrained_gguf(
    "./content/drive/MyDrive/outputs",
    tokenizer=tokenizer,
    quantization_method=quantization_method,
)

Unsloth: You have 1 CPUs. Using `safe_serialization` is 10x slower.
We shall switch to Pytorch saving, which will take 3 minutes and not 30 minutes.
To force `safe_serialization`, set it to `None` instead.
Unsloth: Kaggle/Colab has limited disk space. We need to delete the downloaded
model which will save 4-16GB of disk space, allowing you to save on Kaggle/Colab.
Unsloth: Will remove a cached repo with size 2.2G


Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 5.91 out of 12.67 RAM for saving.


100%|██████████| 26/26 [00:02<00:00, 12.70it/s]


Unsloth: Saving tokenizer... Done.
Unsloth: Saving model... This might take 5 minutes for Llama-7b...
Unsloth: Saving ./content/drive/MyDrive/outputs/pytorch_model-00001-of-00002.bin...
Unsloth: Saving ./content/drive/MyDrive/outputs/pytorch_model-00002-of-00002.bin...
Done.


Unsloth: Converting gemma2 model. Can use fast conversion = False.


==((====))==  Unsloth: Conversion from QLoRA to GGUF information
   \\   /|    [0] Installing llama.cpp will take 3 minutes.
O^O/ \_/ \    [1] Converting HF to GGUF 16bits will take 3 minutes.
\        /    [2] Converting GGUF 16bits to ['q4_k_m'] will take 10 minutes each.
 "-____-"     In total, you will have to wait at least 16 minutes.

Unsloth: [0] Installing llama.cpp. This will take 3 minutes...
Unsloth: [1] Converting model at ./content/drive/MyDrive/outputs into f16 GGUF format.
The output location will be ././content/drive/MyDrive/outputs/unsloth.F16.gguf
This will take 3 minutes...
INFO:hf-to-gguf:Loading model: outputs
INFO:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
INFO:hf-to-gguf:Exporting model...
INFO:hf-to-gguf:gguf: loading model weight map from 'pytorch_model.bin.index.json'
INFO:hf-to-gguf:gguf: loading model part 'pytorch_model-00001-of-00002.bin'
INFO:hf-to-gguf:token_embd.weight,                 torch.float16 --> F16, shape = {2304, 256000}
I

In [ ]:
# Hub 에 GGUF 업로드
model.push_to_hub_gguf(
    "Envy1025/math_teachear_mingle",
    tokenizer,
    quantization_method=quantization_method,
    token=hf_token,
)

Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 6.47 out of 12.67 RAM for saving.


100%|██████████| 26/26 [00:00<00:00, 35.12it/s]


Unsloth: Saving tokenizer... Done.
Unsloth: Saving model... This might take 5 minutes for Llama-7b...
Unsloth: Saving Envy1025/math_teachear_mingle/pytorch_model-00001-of-00002.bin...
Unsloth: Saving Envy1025/math_teachear_mingle/pytorch_model-00002-of-00002.bin...
Done.
==((====))==  Unsloth: Conversion from QLoRA to GGUF information
   \\   /|    [0] Installing llama.cpp will take 3 minutes.
O^O/ \_/ \    [1] Converting HF to GGUF 16bits will take 3 minutes.
\        /    [2] Converting GGUF 16bits to ['q4_k_m'] will take 10 minutes each.
 "-____-"     In total, you will have to wait at least 16 minutes.

Unsloth: [0] Installing llama.cpp. This will take 3 minutes...
Unsloth: [1] Converting model at Envy1025/math_teachear_mingle into f16 GGUF format.
The output location will be ./Envy1025/math_teachear_mingle/unsloth.F16.gguf
This will take 3 minutes...
INFO:hf-to-gguf:Loading model: math_teachear_mingle
INFO:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
INFO:hf-to-

unsloth.F16.gguf:   0%|          | 0.00/5.24G [00:00<?, ?B/s]

No files have been modified since last commit. Skipping to prevent empty commit.


Saved GGUF to https://huggingface.co/Envy1025/math_teachear_mingle
Unsloth: Uploading GGUF to Huggingface Hub...


unsloth.Q4_K_M.gguf:   0%|          | 0.00/1.71G [00:00<?, ?B/s]

No files have been modified since last commit. Skipping to prevent empty commit.


Saved GGUF to https://huggingface.co/Envy1025/math_teachear_mingle
